# 1. Learning the XOR

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Import data

In [2]:
# xor task
xor_data = np.array([[1, 0],[0, 1],[1, 1],[0, 0]])
xor_label = np.array([[1], [1], [0], [0]])

## Build graph 

In [3]:
with tf.Graph().as_default() as g:
    # Data
    x_input = tf.placeholder(tf.float32, [None, 2],name = 'X')
    y_label = tf.placeholder(tf.float32, [None, 1],name = 'Y')
    # Weights
    W = tf.Variable(tf.truncated_normal([2,2]), name = "W")
    w = tf.Variable(tf.truncated_normal([2,1]), name = "w")
    # Bias
    Bias1 = tf.Variable(tf.zeros([4,2]), name = "Bias1")
    Bias2 = tf.Variable(tf.zeros([4,1]), name = "Bias2")

    A2 = tf.sigmoid(tf.matmul(x_input, W) + Bias1)
    y_estimate = tf.sigmoid(tf.matmul(A2, w) + Bias2)

    with tf.name_scope("loss") as scope:
        loss = -tf.reduce_mean(y_label * tf.log(y_estimate) + (1 - y_label) * tf.log(1 - y_estimate))

    with tf.name_scope("train") as scope:
        train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    with tf.name_scope("accuracy") as scope:
        predicted = tf.cast(y_estimate > 0.5, dtype=tf.float32)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_label), dtype=tf.float32))
    
    init = tf.global_variables_initializer()

## Build session 

In [4]:
with tf.Session(graph=g) as sess:
    sess.run(init)
    for epoch in range(501):
        sess.run(train_step, feed_dict={x_input: xor_data, y_label: xor_label})
        if epoch == 500:
            print("_"*80)
            print('Epoch: ', epoch)
            print('   y_estimate: ')
            for element in sess.run(y_estimate, feed_dict={x_input: xor_data, y_label: xor_label}):
                print('    ',element)
            print('\n   Weights: ')
            for element in sess.run(W):
                print('    ',element)
            print('\n   Bias: ')
            for element in sess.run(Bias1):
                print('    ',element)  
            print('\n   Accuracy: ', sess.run(accuracy, feed_dict={x_input:xor_data,y_label:xor_label}))
    print("_"*80)
    writer = tf.summary.FileWriter("./logs/xor_logs", sess.graph)

________________________________________________________________________________
Epoch:  500
   y_estimate: 
     [0.5550531]
     [0.55664515]
     [0.31437135]
     [0.2302585]

   Weights: 
     [-1.1192467  -0.70653313]
     [ 0.21173115 -0.8199856 ]

   Bias: 
     [-0.03007021 -0.24647555]
     [-0.0408465  -0.23831668]
     [0.02086242 0.11747056]
     [0.01764813 0.1269448 ]

   Accuracy:  1.0
________________________________________________________________________________


# 2. MNIST - Digits classification

## Import data

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
# load mnist data
mnist = input_data.read_data_sets("data/mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Set parameters

In [6]:
# Parameters
epochs = 100
batch_size = 100
learning_rate = 0.001

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

## Define fully connected layer

In [7]:
# load mnist data
mnist = input_data.read_data_sets("data/mnist", one_hot=True)

# weight and bais wrappers
def weight_variable(name, shape):
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)
def bias_variable(name, shape):
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)

def fully_connected_layer(x, num_units, name, use_relu=True):
    in_dim = x.get_shape()[1]
    W = weight_variable(name, shape=[in_dim, num_units])
    b = bias_variable(name, [num_units])
    layer = tf.matmul(x, W)
    layer += b
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

Extracting data/mnist\train-images-idx3-ubyte.gz
Extracting data/mnist\train-labels-idx1-ubyte.gz
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


## Build graph 

In [8]:
with tf.Graph().as_default() as g2: 
    # data
    x = tf.placeholder(tf.float32, shape=[None, n_input], name='X')
    y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')
    
    # model
    fc1 = fully_connected_layer(x, n_hidden_1, 'FC1', use_relu=True)
    output_logits = fully_connected_layer(fc1, n_classes, 'OUT', use_relu=False)

    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output_logits), 
                              name='loss')
    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
    
    # accuracy
    cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')
    correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    
    init = tf.global_variables_initializer()

## Build session ( validation result )

In [9]:
with tf.Session(graph=g2) as sess:
    sess.run(init)
    global_step = 0
    # Number of training iterations in each epoch
    num_tr_iter = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        for iteration in range(num_tr_iter):
            global_step += 1
            start = iteration * batch_size
            end = (iteration + 1) * batch_size
            x_batch, y_batch = mnist.train.next_batch(batch_size)

            # Run optimization op (backprop)
            feed_dict_batch = {x: x_batch, y: y_batch}
            sess.run(optimizer, feed_dict=feed_dict_batch)

        # Run validation after every epoch
    feed_dict_valid = {x: mnist.validation.images, y: mnist.validation.labels}
    loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print("[Validation result]")
    print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".
          format(epoch + 1, loss_valid, acc_valid))

[Validation result]
Epoch: 100, validation loss: 0.12, validation accuracy: 98.1%


## Build session ( testing data result )

In [10]:
with tf.Session(graph=g2) as sess:
    sess.run(init)
    global_step = 0
    # Number of training iterations in each epoch
    num_tr_iter = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        for iteration in range(num_tr_iter):
            global_step += 1
            start = iteration * batch_size
            end = (iteration + 1) * batch_size
            x_batch, y_batch = mnist.train.next_batch(batch_size)

            # Run optimization op (backprop)
            feed_dict_batch = {x: x_batch, y: y_batch}
            sess.run(optimizer, feed_dict=feed_dict_batch)

    # Run testing data after every epoch
    feed_dict_test = {x: mnist.test.images, y: mnist.test.labels}
    loss_test, acc_test = sess.run([loss, accuracy], feed_dict=feed_dict_test)
    print("[Testing result]")
    print("Epoch: {0}, testing loss: {1:.2f}, testing accuracy: {2:.01%}".
          format(epoch + 1, loss_test, acc_test))

[Testing result]
Epoch: 100, testing loss: 0.13, testing accuracy: 98.2%
